In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import Hamiltonian as H
import K_sum as K
import cal_parameter as param
np.set_printoptions(threshold=784,linewidth=np.inf)

In [21]:
#####################Local Hamiltonian##########################

def Local_odd(gamma, n):
    '''Independent Function, Creates H_0 Hamiltonian in sine basis,
        n is dimensinon of matrix'''

    A = [[0 for j in range(n)] for k in range(n)]

    for i in range(n):
        for j in range(n):
            try:
                if i==j:
                    A[i][j] = ((i+1)**2)
                if abs(i-j) == 1:
                    A[i][j] = -gamma/2
                else:
                    None
            except:
                None

    return np.array(A)

def Local_odd_Eigenvec(gamma: float, n: int ,m: int):
    ''' Requires Local_odd, m is mth eigenvector of H_0 Matrix, corresponds with mth eigenvalue'''

    A = Local_odd(gamma,n)
    A_eigvec = np.linalg.eig(A)
    A_trans = np.transpose(A_eigvec[1])

    return A_trans[m]

def Local_odd_Eigenval(gamma: float,n: int,m: int):
    ''' Requires Local_odd, m it mth eigenvalue of H_0 Matrix, corresponds with mth eigenvector'''

    A = Local_odd(gamma,n)
    A_eigval = np.linalg.eig(A)

    return A_eigval[0][m]

def Local_even(gamma:float, n:int):
    '''Independent Function, Creates H_0 Hamiltonian in cosine basis
    n = dimension of matrix'''

    A = [[0 for j in range(n)] for k in range(n)]

    for i in range(n):
        for j in range(n):
            try:
                if i==j:
                    A[i][j] = (i)**2
                if abs(i-j) == 1:
                    A[i][j] = -gamma/2
                else:
                    None
            except:
                None

    np_A = np.array(A)
    np_A[0][1] = -gamma/(np.sqrt(2))
    np_A[1][0] = -gamma/(np.sqrt(2))

    return np_A

def Local_even_Eigenvec(gamma: float,n: int ,m: int):
    ''' Requires Local_even, m is mth eigenvector of H_0 Matrix, corresponds with mth eigenvalue'''

    A = Local_even(gamma,n)
    A_eig = np.linalg.eig(A)
    A_trans = np.transpose(A_eig[1])

    return A_trans[m]

def Local_even_Eigenval(gamma: float,n: int,m: int):
    ''' Requires Local_even, m it mth eigenvalue of H_0 Matrix, corresponds with mth eigenvector'''

    A = Local_even(gamma,n)
    A_eig = np.linalg.eig(A)

    return A_eig[0][m]

def Elements(d,x,y):

    x_trans = np.transpose(x)

    arr = np.matmul(d,y)
    arr2 = np.matmul(x_trans,arr)

    return arr2

def INT_odd_Eigenvec(gamma, n, m):
    LOC_VEC_O = Local_odd_Eigenvec(gamma, n, m)
    INT_ARR_O = []

    for i in range(n):
        if i == 0:
            INT_ARR_O.append(0)
        else:
            INT_ARR_O.append(-1j * i * LOC_VEC_O[i-1])
    
    return np.array(INT_ARR_O)

def Local_For_INT_odd_Eigenvec(gamma,n,m):
    LOC_VEC_O = Local_odd_Eigenvec(gamma,n,m)
    INT_ARR_O = []

    for i in range(n):
        if i==0:
            INT_ARR_O.append(0)
        else:
            INT_ARR_O.append(LOC_VEC_O[i-1])

    return np.array(INT_ARR_O)

def INT_even_Eigenvec(gamma, n, m):
    LOC_VEC_E = Local_even_Eigenvec(gamma, n, m)
    INT_ARR_E = []

    for i in range(n):
        INT_ARR_E.append(1j * i * LOC_VEC_E[i]) # Once differentiat, minus sign appears, and vanished because of the factor (-i)

    return np.array(INT_ARR_E)

#################### Main Hamiltonian ##########################

def Hamiltonian_Matrix(gamma: float,n: int, g: float,omega: float):
    '''Return Hamiltonian H in matrix form, requires prerequisites,
    r : Value of Gamma, z : dimension of Matrix, g : coupling strength, omega : frequency '''

    ######### Elements of Hamiltonian matrix ###############
    
    # Eigenvalues for local and bath part
    LOC_EV_ODD = Local_odd_Eigenval(gamma,3,0)
    LOC_EV_EVE_g = Local_even_Eigenval(gamma,3,0)
    LOC_EV_EVE_s = Local_even_Eigenval(gamma,3,1)

    # Eigenvectors for N matrix
    LOC_ODD = Local_For_INT_odd_Eigenvec(gamma,3,0)

    N_EVE_g = INT_even_Eigenvec(gamma, 3, 0)
    N_EVE_s = INT_even_Eigenvec(gamma, 3, 1)

    N_ELE_10 = np.dot(LOC_ODD,N_EVE_g)
    N_ELE_01 = -N_ELE_10 #first state dot product to ground state
    N_ELE_12 = np.dot(LOC_ODD,N_EVE_s)
    N_ELE_21 = -N_ELE_12
    

    # N_Matrix (interaction Hamiltonian)
    A = [[0 for i in range(n)] for j in range(n)]

    for i in range (n):
        for j in range (n):
            try:
                if i==j and i%3 == 0:
                   A[i][j+4] = (np.sqrt((j+4)//3)) * g * N_ELE_01
                if i==j and i%3 == 1:
                    A[i][j+2] = (np.sqrt((j+2)//3)) * g * N_ELE_10
                    A[i][j+4] = (np.sqrt((j+4)//3)) * g * N_ELE_12

                if i==j and i%3 == 2:
                    A[i][j+2] = (np.sqrt((j+2)//3)) * g * N_ELE_21
                
            except:
                None

    for i in range (n):
        for j in range (n):
            try:
                if i==j and j%3 == 0:
                    A[j+4][j] = np.sqrt((i+3)//3) * g * N_ELE_10
                if i==j and j%3 == 1:
                    A[j+2][j] = np.sqrt((i+2)//3) * g * N_ELE_01
                    A[j+4][j] = np.sqrt((i+4)//3) * g * N_ELE_21

                if i==j and j%3 == 2:
                    A[j+2][j] = np.sqrt((i+2)//3) * g * N_ELE_12
                
            except:
                None


    np_A = np.array(A)
    #np_B = np.transpose(np_A.copy())
    
    # Hamiltonian_local , Hamiltonian_bath
    for i in range(n):
        for j in range(n):
            try:
                if i==j and i%3==0:
                    A[i][j] = LOC_EV_EVE_g + (j//3)
                elif i==j and i%3==1:
                    A[i][j] = LOC_EV_ODD + (j//3)
                elif i==j and i%3==2:
                    A[i][j] = LOC_EV_EVE_s + (j//3)
                else:
                    A[i][j] = 0
            except:
                None
    
    np_C = np.array(A)
    Array = np_A + np_C

    return Array

def Hamiltonian_Matrix_Eigenval(gamma,n,g,omega,i):
    A_eigval = np.linalg.eigh(Hamiltonian_Matrix(gamma,n,g,omega))[0][i] # return value of eigh[0] : eigenvalues of corresponding eigenvectors
    return A_eigval

def Hamiltonian_Matrix_Eigenvec(gamma,n,g,omega,i):
    A = np.linalg.eigh(Hamiltonian_Matrix(gamma,n,g,omega))[1].T # return value of eigh[1] : eigenvectors of corresponding eigenvalues
    A_eigvec = A[i]
    return A_eigvec

## Lie group ####################################
def Lie_group(x):
    sigma = [[0 for i in range(3)] for j in range(3)]

    if x == 1:
        sigma[0][1] = 1
        sigma[1][0] = 1
        return np.array(sigma)
    if x == 2:
        sigma[0][1] = 1j
        sigma[1][0] = -1j
        return np.array(sigma)
    if x == 3:
        sigma[0][0] = 1
        sigma[1][1] = 1
        return np.array(sigma)
    if x == 4:
        sigma[0][2] = 1
        sigma[2][0] = 1
        return np.array(sigma)
    if x == 5:
        sigma[0][2] = -1j
        sigma[2][0] = 1j
        return np.array(sigma)
    if x == 6:
        sigma[1][2] = 1
        sigma[2][1] = 1
        return np.array(sigma)
    if x == 7:
        sigma[1][2] = -1j
        sigma[2][1] = 1j
        return np.array(sigma)
    if x == 8:
        sigma[0][0] = 1/(3**0.5)
        sigma[2][2] = -2/(3**0.5)
        return np.array(sigma)

def Lie_tensorproduct(x: int,y: int):
    '''x = select the xth Lie group element, y = dimension of Identity matrix'''
    A = np.identity(y)
    Lie_Tens = np.kron(A,Lie_group(x))
    return Lie_Tens
## Lie group ####################################



In [22]:
def Chi_sp(beta : float, gamma : float, omega: float, g: float, n: int , tau: float):
    '''Chi_function, r : value of gamma, z : dimension/3 of hilbert space, g : coupling strength, omega : frequency'''

    # Tr Z
    Exp_val = sp.linalg.expm(-beta*Hamiltonian_Matrix(gamma,3*n,g,omega))
    Z = np.trace(Exp_val)

    # Chi_calculation
    A = []
    for i in range(3*n):
        for j in range(3*n):
            E_N = Hamiltonian_Matrix_Eigenval(gamma,3*n,g,omega,i)
            E_M = Hamiltonian_Matrix_Eigenval(gamma,3*n,g,omega,j)
            Exp_val_chi = np.exp(-(beta-tau) * E_N - tau * E_M)

            Expec_NM = np.conjugate(Hamiltonian_Matrix_Eigenvec(gamma,3*n,g,omega,i)) @ Lie_tensorproduct(1,n) @ Hamiltonian_Matrix_Eigenvec(gamma,3*n,g,omega,j)
            Expec = Expec_NM * np.conjugate(Expec_NM)


            A.append(Exp_val_chi * Expec)
    
    #print(A)
    # Chi_total
    np_A = np.array(A)
    sum_A = np.sum(np_A)

    return sum_A/Z

In [26]:
betamaxarray = [1]

'''
betamaxarray = [0 for i in range(5)]

for j in range(1,len(betamaxarray),1):
    betamaxarray[j] = round(0.5 + betamaxarray[j-1],2)
    betamaxarray[j-1] = betamaxarray[j]
'''

'''
for j in range(11):
    if j == 0:
        betamaxarray[j] = 0.001
    if j > 0:
        betamaxarray[j] = round(j * 0.1,1)
'''

for b in betamaxarray:

    ta = np.linspace(0,b,201)
    tau = np.linspace(0,b,201)
    k = np.full(100,1)
    
    
    garray = [0 for i in range(25)]
    gsquare = [0 for i in range(len(garray))]
    for i in range(len(garray)):
        if (i<20):
            garray[i+1] = round(garray[i]+0.05,2)
            gsquare[i+1] = round(garray[i+1]**2,4)
        elif (i<24):
            gsquare[i+1] = round(gsquare[i] + 0.2,1)
    
'''
    garray = [0 for i in range(25)]
    gsquare = [0 for i in range(30)]
    gsquare[0] = 1
'''
g_arr = [0 for i in range(21)]

'''
    for i in range(30):
        if (i<20):
            garray[i+1] = round(garray[i]+0.05,2)
            gsquare[i+1] = round(garray[i+1]**2,4)
        if (i>=20):
            gsquare[i] = gsquare[i-1] + 0.2
'''

for i in range(21):
    if i==0:
        g_arr[i] = i
    if i>0:
        g_arr[i] = round(g_arr[i-1]+0.01,2)


def plotting(r,g,beta):
    A =[]
    for i in range(1):
        plot = Chi_sp(beta,r,1,g,10,ta[len(ta)-1])
        A.append(plot)

    return(A)

value = []
domain = []
for i in gsquare:
    value.append(plotting(1,i,b))
    val = np.array(value)

domain = g_square
df = np.column_stack((domain,val.real,val.imag))

np.savetxt('Exacttest_grid201_size30_highg_tau_0.txt',df)

NameError: name 'g_square' is not defined

In [17]:
import numpy as np
import scipy as sp
import cal_parameter as param
import Hamiltonian as H
import K_sum as K

g_arr = [0 for i in range(21)]

for i in range(21):
    if i==0:
        g_arr[i] = i
    if i>0:
        g_arr[i] = round(g_arr[i-1] + 0.01,2)

for i in g_arr:
    HM = H.Hamiltonian_Matrix(param.gamma,6,i,1)

    domain = param.tau_grid
    value = H.Chi_sp(HM,param.tau_grid)

    df = np.column_stack((domain.real,value.real,value.imag))

    np.savetxt('Exacttest_beta1_g_{}.txt'.format(i),df)

In [18]:
g_arr

[0,
 0.01,
 0.02,
 0.03,
 0.04,
 0.05,
 0.06,
 0.07,
 0.08,
 0.09,
 0.1,
 0.11,
 0.12,
 0.13,
 0.14,
 0.15,
 0.16,
 0.17,
 0.18,
 0.19,
 0.2]